In [14]:
import geopandas as gpd
import pandas as pd
import os
import fiona

In [15]:
carpeta_principal = r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas"
archivos_a_excluir = ["CCNN_PTOSINTERES.shp"]

In [16]:
gdf_list = []
for raiz, _, archivos in os.walk(carpeta_principal):
    for archivo in archivos:
        ruta_completa = os.path.join(raiz, archivo)

        # Ignorar archivos específicos
        if archivo in archivos_a_excluir:
            print(f"Ignorando archivo: {archivo}")
            continue  

        # Leer archivos SHP
        if archivo.endswith(".shp"):
            try:
                gdf = gpd.read_file(ruta_completa)
                
                # Convertir CRS si es diferente
                if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
                    gdf = gdf.to_crs("EPSG:4326")
                
                gdf["source"] = ruta_completa
                gdf_list.append(gdf)
            except Exception as e:
                print(f"Error al leer {ruta_completa}: {e}")

        # Leer archivos KML
        elif archivo.endswith(".kml"):
            try:
                # Obtener la lista de capas en el archivo KML
                layers = fiona.listlayers(ruta_completa)
                
                # Intentar encontrar la capa que más se parece al nombre del archivo
                expected_layer = archivo[:-4]  # Nombre esperado
                matched_layer = None
                
                for layer in layers:
                    if expected_layer in layer:
                        matched_layer = layer
                        break
                
                if matched_layer is None:
                    print(f"No se encontró una capa coincidente en {ruta_completa}, usando la primera disponible.")
                    matched_layer = layers[0]  # Usar la primera capa si no hay coincidencias

                # Leer el archivo KML con la capa correcta
                gdf = gpd.read_file(ruta_completa, driver='KML', layer=matched_layer)

                # Convertir CRS si es diferente
                if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
                    gdf = gdf.to_crs("EPSG:4326")

                gdf["source"] = ruta_completa
                gdf_list.append(gdf)

            except Exception as e:
                print(f"Error al leer {ruta_completa}: {e}")


Ignorando archivo: CCNN_PTOSINTERES.shp
No se encontró una capa coincidente en C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\Rutas Parques Naturales\015-Archipielago_de_Cabrera\15-235-Excursion7-Ruta_del_faro_de_lEnciola.kml, usando la primera disponible.
No se encontró una capa coincidente en C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\Rutas Parques Naturales\07-Aiguestortes_i_Estany_de_Sant_Maurici\7-165-PRPNSN3-Hacia_las_Agulles_dAmitges.kml, usando la primera disponible.


In [17]:
# Unificar todos los GeoDataFrames manteniendo todas las columnas (rellenando valores faltantes con NaN)
gdf_unificado = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True, join='outer'))

# Ver las primeras filas para comprobar que todo está correcto
gdf_unificado.head()

,ID,NOM_CAMINO,TIPOLOGIA,NOM_ETAPA,LONGITUD,ETAPA,CCAA,PROVINCIA,HUSO,ID_SECTOR,...,nombre,longitud,fecha_edi,recorrido,provincia,comunidad,estado,firme,nombre_en,fecha_edit
0,778.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 1: La Esperanza - La Caldera,30.924081,1,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,779.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 2: La Caldera - El Portillo,14.171262,2,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,780.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 3: El Portillo - Parador de Las Cañadas ...,16.686617,3,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,781.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 4: Parador de Las Cañadas del Teide - Vi...,15.420307,4,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,782.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 5: Vilaflor - Arona,18.017360,5,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Definir la ruta de salida
ruta_salida = os.path.join(carpeta_principal, "archivo_unificado.shp")

# Guardar el GeoDataFrame en un archivo SHP
gdf_unificado.to_file(ruta_salida)

print(f"Archivo unificado guardado en: {ruta_salida}")

C:\Users\maria.b.calvo\AppData\Local\Temp\ipykernel_35368\10481967.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_unificado.to_file(ruta_salida)
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'id' to 'id_1'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'longitud' to 'longitud_1'
  ogr_write(
C:\Users\maria.b.calvo\AppData\

Archivo unificado guardado en: C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_unificado.shp


In [19]:
# Imprimir nombres de las columnas (atributos)
print("Columnas del SHP:", gdf_unificado.columns)

Columnas del SHP: Index(['ID', 'NOM_CAMINO', 'TIPOLOGIA', 'NOM_ETAPA', 'LONGITUD', 'ETAPA',
       'CCAA', 'PROVINCIA', 'HUSO', 'ID_SECTOR', 'ID_CAMINO', 'COD_CAMINO',
       'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO', 'NOM_TOTAL', 'XCENTROIDE',
       'YCENTROIDE', 'geometry', 'source', 'Name', 'Description', 'id',
       'nombre', 'longitud', 'fecha_edi', 'recorrido', 'provincia',
       'comunidad', 'estado', 'firme', 'nombre_en', 'fecha_edit'],
      dtype='object')


In [20]:
gdf_atributos = gdf_unificado.copy()

# Diccionario de columnas equivalentes
column_mapping = {
    "Nombre_Ruta": ["NOM_CAMINO", "Name", "nombre"],
    "Longitud": ["LONGITUD", "longitud"],
    "Provincia": ["PROVINCIA", "provincia"],
    "CC_AA": ["CCAA", "comunidad"]
}

# Crear nuevas columnas con los valores existentes
for new_col, old_cols in column_mapping.items():
    gdf_atributos[new_col] = gdf_atributos[old_cols].bfill(axis=1).iloc[:, 0]  # Rellenar con el primer valor no nulo

# Eliminar las columnas antiguas si ya no se necesitan
columns_to_drop = sum(column_mapping.values(), [])  # Convertir listas de valores en una sola lista
gdf_atributos = gdf_atributos.drop(columns=columns_to_drop, errors="ignore")

# Imprimir nombres de las columnas (atributos)
print("Columnas del SHP:", gdf_atributos.columns)

Columnas del SHP: Index(['ID', 'TIPOLOGIA', 'NOM_ETAPA', 'ETAPA', 'HUSO', 'ID_SECTOR',
       'ID_CAMINO', 'COD_CAMINO', 'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO',
       'NOM_TOTAL', 'XCENTROIDE', 'YCENTROIDE', 'geometry', 'source',
       'Description', 'id', 'fecha_edi', 'recorrido', 'estado', 'firme',
       'nombre_en', 'fecha_edit', 'Nombre_Ruta', 'Longitud', 'Provincia',
       'CC_AA'],
      dtype='object')


In [21]:
# Definir la ruta de salida
ruta_salida = os.path.join(carpeta_principal, "archivo_atributos_unificados.shp")

# Guardar el GeoDataFrame en un archivo SHP
gdf_atributos.to_file(ruta_salida)

C:\Users\maria.b.calvo\AppData\Local\Temp\ipykernel_35368\918135260.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_atributos.to_file(ruta_salida)
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'id' to 'id_1'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre_Ruta' to 'Nombre_Rut'
  ogr_write(
C:\Users\maria.b.calvo\AppD